In [2]:
import warnings
warnings.filterwarnings('ignore')

In [24]:
import pythainlp
from pythainlp import spell
from pythainlp.spell import NorvigSpellChecker
from pythainlp.corpus import ttc  # Thai Textbook Corpus
from pythainlp import correct
from pythainlp.util import normalize
from pythainlp import thai_tonemarks

In [8]:
import codecs
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re

In [5]:
file_name="02+03_dup_clean" # ชื่อไฟล์คลังข้อมูล

In [9]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
 text=re.sub("<[^>]*>","",p)
 text=re.sub("\[(.*?)\]","",text)
 text=re.sub("\[\/(.*?)\]","",text)
 if text not in data_not:
  data_not.append(text)
  return True
 else:
  return False
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
 text=text.strip()
 text=re.sub("<[^>]*>","",text)
 text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')#  ตัดการกับพวกไม่มี tag word
 text=re.sub("(\[\w+\])","***\\1",text)
 text2=[]
 for i in text.split('***'):
  if "[" in i:
   text2.append(i)
  else:
   text2.append("[word]"+i+"[/word]")
 text="".join(text2)#re.sub("[word][/word]","","".join(text2))
 return text.replace("[word][/word]","")
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text) # นำไปใส่ tag [word]
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text) # แยก tag ออกมาจากข้อความ
    j=0
    conll2002="" # ประกาศตัวแปรเก็บ conll2002
    for tagopen,text,tagclose in tag: # ลูปใน tag โดยเป็น (tagopen,text,tagclose)
        word_cut=word_tokenize(text,engine="newmm") # ใช้ตัวตัดคำ newmm ของ PyThaiNLP
        i=0
        txt5=""
        while i<len(word_cut): #ลูปตามจำนวน token ที่ตัดในtag
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word': # ไม่เป็น tag [word] และเป็น i หรือตัวเริ่มต้น tag
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else: # เป็น [word]
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002) # เพิ่ม postag ใส่
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
	"""
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
	with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
		lines = f.read().splitlines()
	return [a for a in lines if Unique(a)] # เอาไม่ซ้ำกัน

def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n') # นำไปแปลงเป็น conll2002
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
	string=""
	for data in lists:
		string1=""
		for j in data:
			string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
		string1+="\n"
		string+=string1
	return string

def get_data_tag(listd):
	list_all=[]
	c=[]
	for i in listd:
		if i !='':
			c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
		else:
			list_all.append(c)
			c=[]
	return list_all
def getall(lista):
    ll=[]
    for i in lista:
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

In [10]:
data1=getall(get_data(file_name+".txt")) # นำคลังเข้าไป แยกออกเป็น list ละบรรทัด
datatofile=alldata_list(data1) # นำไปผ่านขั้นตอน 1 2 3 4
datatofile[0]

[('มา', 'XVAE', 'O'),
 ('แย้', 'VACT', 'O'),
 ('วววววววววววววววววววววว', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('กับ', 'RPRE', 'O'),
 ('รีวิว', 'NCMN', 'O'),
 ('ง่ายๆ', 'VATT', 'O'),
 (' ', 'PUNC', 'O'),
 ('เห็นภาพ', 'NCMN', 'O'),
 ('สดๆ', 'VSTA', 'O'),
 ('กับ', 'RPRE', 'O'),
 ('กล้อง', 'NCMN', 'O'),
 ('โทรศัพท์', 'NCMN', 'O'),
 ('ธรรมดาๆ', 'NCMN', 'O'),
 ('นี่แหละ', 'NCMN', 'O'),
 ('ครับ', 'NCMN', 'O'),
 ('   ', 'NCMN', 'O'),
 ('3', 'NCNM', 'O'),
 (' ', 'PUNC', 'O'),
 ('-', 'PUNC', 'O'),
 (' ', 'PUNC', 'O'),
 ('4', 'DCNM', 'O'),
 (' ', 'PUNC', 'O'),
 ('FEB', 'NCMN', 'O'),
 (' ', 'PUNC', 'O'),
 ('2018', 'NCNM', 'O'),
 ('   ', 'NCMN', 'O'),
 ('มา', 'XVAM', 'O'),
 ('เริ่ม', 'VSTA', 'O'),
 ('กัน', 'ADVN', 'O'),
 ('เลย', 'XVAE', 'O'),
 ('โจทย์', 'NCMN', 'O'),
 ('ใน', 'RPRE', 'O'),
 ('ทริป', 'NCMN', 'O'),
 ('นี้', 'DDAC', 'O'),
 ('ขอ', 'VACT', 'O'),
 ('บรรยากาศ', 'NCMN', 'O'),
 ('แบบ', 'NCMN', 'O'),
 ('ฟินๆ', 'NCMN', 'O'),
 ('นั่ง', 'NCMN', 'O'),
 ('แช่', 'VACT', 'O'),
 ('สระน้ำ', 'NCMN', 'O'

In [23]:
def remove_smallpiyan(word):
    return False if "ฯ" in word else True

checker_custom_filter = NorvigSpellChecker(dict_filter=remove_smallpiyan)  # use custom filter
len(checker_custom_filter.dictionary())

66207

In [25]:
_NORMALIZE_RULE1 = [
    "ะ",
    "ั",
    "็",
    "า",
    "ิ",
    "ี",
    "ึ",
    "่",
    "ํ",
    "ุ",
    "ู",
    "ใ",
    "ไ",
    "โ",
    "ื",
    "่",
    "้",
    "๋",
    "๊",
    "ึ",
    "์",
    "๋",
    "ำ",
]  # เก็บพวกสระ วรรณยุกต์ที่ซ้ำกันแล้วมีปัญหา

_NORMALIZE_RULE2 = [
    ("เเ", "แ"),  # เ เ -> แ
    ("ํ(t)า", "\\1ำ"),
    ("ํา(t)", "\\1ำ"),
    ("([่-๋])([ัิ-ื])", "\\2\\1"),
    ("([่-๋])([ูุ])", "\\2\\1"),
    ("ำ([่-๋])", "\\1ำ"),
    ("(์)([ัิ-ื])", "\\2\\1"),
]  # เก็บพวก พิมพ์ลำดับผิดหรือผิดแป้นแต่กลับแสดงผลถูกต้อง ให้ไปเป็นแป้นที่ถูกต้อง เช่น เ + เ ไปเป็น แ


def normalize(text: str) -> str:
    """
    This function normalize thai text with normalizing rules as follows:

        * Remove redudant symbol of tones and vowels.
        * Subsitute ["เ", "เ"] to "แ".

    :param str text: thai text to be normalized
    :return: normalized Thai text according to the fules
    :rtype: str

    :Example:

        >>> from pythainlp.util import normalize
        >>>
        >>> normalize('สระะน้ำ')
        สระน้ำ
        >>> normalize('เเปลก')
        แปลก
        >>> normalize('นานาาา')
        นานา
    """
    for data in _NORMALIZE_RULE2:
        text = re.sub(data[0].replace("t", "[่้๊๋]"), data[1], text)
    for data in list(zip(_NORMALIZE_RULE1, _NORMALIZE_RULE1)):
        text = re.sub(data[0].replace("t", "[่้๊๋]") + "+", data[1], text)
    return text



def deletetone(text: str) -> str:
    """
    This function removes Thai tonemarks from the text.
    There are 4 tonemarks indicating 4 tones as follows:

        * Down tone (Thai: ไม้เอก  _่ )
        * Falling tone  (Thai: ไม้โท  _้ )
        * High tone (Thai: ไม้ตรี  ​_๊ )
        * Rising tone (Thai: ไม้จัตวา _๋ )

    :param str text: text in Thai language
    :return: text without Thai tonemarks
    :rtype: str

    :Example:

        >>> from pythainlp.util import deletetone
        >>>
        >>> deletetone(\\
            'สองพันหนึ่งร้อยสี่สิบเจ็ดล้านสี่แสนแปดหมื่นสามพันหกร้อยสี่สิบเจ็ด')
        สองพันหนึงรอยสีสิบเจ็ดลานสีแสนแปดหมืนสามพันหกรอยสีสิบเจ็ด
    """
    chars = [ch for ch in text if ch not in thai_tonemarks]
    return "".join(chars)

In [ ]:
for i in range(len(datatofile)):
    for j in range(len(datatofile[i])):
        normalize(datatofile[i][j][0])
        deletetone(datatofile[i][j][0])
        correct(datatofile[i][j][0])

In [ ]:
datatofile

In [ ]:
f=open("datatofile.txt","w")
f.write(datatofile)